In [1]:
%%time
print("starting")
#import in all libraries and methods
%run -i 'myimports.py'
%run -i 'datagathering.py'
%run -i 'nlp_processing.py' #imports packages and methods
%run -i 'timestamp_generation.py'

main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
podcast_host = "Lex Fridman"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"



starting
CPU times: user 15.5 s, sys: 1.27 s, total: 16.8 s
Wall time: 17.9 s


In [ ]:
%%time
doc = nlp(text_fix(open("3Lex/#103|Ben_Goertzel|Artificial_General_Intelligence.txt").read()))
print(doc.user_data.keys())

In [ ]:
update_transcripts(html_location, main)

In [2]:
%%time
podcast_name = "3Lex"

onlyfiles = folder_to_filelist(podcast_name) #get list (complete text, filename)
doc_bin = DocBin(store_user_data=True) #docbin container for serialization
docs = [] #list of docs 
print("starting")
for doc, name in tqdm(nlp.pipe(onlyfiles, as_tuples=True)): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    
    docs.append(doc)
    doc_bin.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())
print("done")

0it [00:00, ?it/s]

starting


/ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
41it [10:15, 15.02s/it]


ValueError: cannot compute LDA over an empty collection (no terms)

In [ ]:
%%time
podcast_name = "5Lex"

onlyfiles1 = folder_to_filelist(podcast_name) #get list (complete text, filename)
doc_bi1n = DocBin(store_user_data=True) #docbin container for serialization
docs1 = [] #list of docs 
print("starting")
for doc, name in tqdm(nlp.pipe(onlyfiles1, as_tuples=True)): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    
    docs1.append(doc)
    doc_bin1.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())
print("done")

In [4]:
for name in listdir("5Lex"):
    for d in docs:
        if (d.user_data["guest"]).replace(" ", "_") in name:
            print(name)
            os.remove("5Lex/"+name)
            break

#140|Lisa_Feldman_Barrett|Love,_Evolution,_and_the_Human_Brain.txt
#144|Michael_Littman|Reinforcement_Learning_and_the_Future_of_AI.txt
#121|Eugenia_Kuyda|Friendship_with_an_AI_Companion.txt
#133|Manolis_Kellis|Biology_of_Disease.txt
#117|Sheldon_Solomon|Death_and_Meaning.txt
#115|Dileep_George|Brain-Inspired_AI.txt
#116|Sara_Seager|Search_for_Planets_and_Life_Outside_Our_Solar_System.txt
#129|Lisa_Feldman_Barrett|Counterintuitive_Ideas_About_How_the_Brain_Works.txt
#77|Alex_Garland|Ex_Machina,_Devs,_Annihilation,_and_the_Poetry_of_Science.txt
#80|Vitalik_Buterin|Ethereum,_Cryptocurrency,_and_the_Future_of_Money.txt
#78|Ann_Druyan|Cosmos,_Carl_Sagan,_Voyager,_and_the_Beauty_of_Science.txt
#104|David_Patterson|Computer_Architecture_and_Data_Storage.txt
#138|Yaron_Brook|Ayn_Rand_and_the_Philosophy_of_Objectivism.txt
#101|Joscha_Bach|Artificial_Consciousness_and_the_Nature_of_Reality.txt
#100|Alexander_Fridman|My_Dad,_the_Plasma_Physicist.txt
#123|Manolis_Kellis|Origin_of_Life,_Humans,_Id

In [5]:
username = urllib.parse.quote_plus('Empyr3an')
password = urllib.parse.quote_plus("B4ldr1c7@1")

cluster = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.4pec2.mongodb.net/Checkra?retryWrites=true&w=majority".format(username, password))
db = cluster["podcasts"]
collection = db["lex"]

# db.create_collection("lmini")

In [6]:
db["lex"].insert_many([doc.user_data for doc in docs])

In [ ]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))
# with open("cucumber/first_doc.bin", "rb") as w: #loading in
#     new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
# docs1 += list(new_docs1.get_docs(nlp.vocab))
for doc in docs1:
    print(doc.user_data["title"], doc.user_data["guest"])
# print(docs1[5].user_data["books"])

In [ ]:
#for each category of ents, generate csv files for ents corresponding to people

att_to_csv(docs1, "people") 
# att_to_csv(docs1, "places")
# att_to_csv(docs1, "books")
# print(sorted(set([item for sublist in docs1 for item in sublist.user_data["people"]])))

In [ ]:
#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['source']=="Matthew Johnson"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['source']=="Matthew Johnson" )| (kg_df['source']=="Ryan Hall")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(kg_df[(kg_df['target']=="Crime and Punishment" )| (kg_df['target']=="Atlas Shrugged")], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()